In [1]:
from collections import OrderedDict
from itertools import product
from pyCBT.data.providers.mt.pairs import get_mt_pairs
from highcharts import Highchart

import ffn

np = ffn.core.np
pd = ffn.data.pd
plt = ffn.core.plt

In [2]:
# define constants
# time frame in periods/minutes (because data is sampled in minutes)
TIME_FRAME = 5*30
# define target price
PRICE = "CLOSE"
# define volume column
VOLUME = "VOLUME"

pairs = get_mt_pairs()

PAIRS = pairs.keys()
CURRENCIES = np.unique([currency for pair in PAIRS for currency in [pair[:3], pair[3:]]])

# extract price for each instrument
price = pd.DataFrame(index=range(5*TIME_FRAME))
for pair_name in PAIRS: price[pair_name] = pairs[pair_name].filter(like=PRICE).values[-5*TIME_FRAME:]

# compute heatmap
H = pd.DataFrame(data=np.nan, index=CURRENCIES, columns=CURRENCIES)
for i, j in product(range(len(CURRENCIES)), range(len(CURRENCIES))):
    if CURRENCIES[i] == CURRENCIES[j]: continue
    pair_name = "{}{}".format(CURRENCIES[j], CURRENCIES[i])

    # compute exchange rate of j/i
    if pair_name in PAIRS:
        ex_rate = price[pair_name]
    else:
        continue
        # get j/USD
        if "USD{}".format(CURRENCIES[j]) in PAIRS:
            price_j = 1.0 / price["USD{}".format(CURRENCIES[j])]
        elif "{}USD".format(CURRENCIES[j]) in PAIRS:
            price_j = price["{}USD".format(CURRENCIES[j])]
        # get i/USD
        if "USD{}".format(CURRENCIES[i]) in PAIRS:
            price_i = 1.0 / price["USD{}".format(CURRENCIES[i])]
        elif "{}USD".format(CURRENCIES[i]) in PAIRS:
            price_i = price["{}USD".format(CURRENCIES[i])]

        ex_rate = price_j / price_i

    H.loc[CURRENCIES[i]][CURRENCIES[j]] = (ex_rate.iloc[-1] - ex_rate.iloc[-TIME_FRAME]) / ex_rate.iloc[-TIME_FRAME] * 100.0

In [7]:
def heatmap_hbar(inp_currency, pairs):
    # filtering
    chart_categories = filter(lambda x: inp_currency in x, pairs)
    chart_dataset = H.filter(like=inp_currency).dropna()

    chart_options = {
        'chart': {
            'type': 'bar'
        },
        'title': {
            'text': '{} heatmap'.format(inp_currency)
        },
        'xAxis': [{
            'categories': chart_categories,
            'reversed': False,
            'labels': {
                'step': 1
            }
        }, { # mirror axis on right side
            'opposite': True,
            'reversed': False,
            'categories': chart_categories,
            'linkedTo': 0,
            'labels': {
                'step': 1
            }
        }],
        'yAxis': {
            'title': {
                'text': None
            },
        },

        'plotOptions': {
            'series': {
                'stacking': 'normal'
            }
        },
        
        'legend': {
            'enabled': False
        },

        'tooltip': {
            'formatter': "function () {\
                return '<b>' + this.point.category + '</b><br/>' +\
                    Highcharts.numberFormat(this.point.y, 2);\
            }"
        }
    }

    dataset_1, dataset_2 = zip(*[(round(heat, 2) if heat < 0.0 else 0.0, round(heat, 2) if heat > 0.0 else 0.0) for heat in chart_dataset.values])

    chart = Highchart()
    chart.set_dict_options(chart_options)
    chart.add_data_set(dataset_1, 'bar')
    chart.add_data_set(dataset_2, 'bar')
    return chart

In [8]:
heatmap_hbar(inp_currency="EUR", pairs=PAIRS)

In [5]:
heatmap_hbar(inp_currency="USD", pairs=PAIRS)

In [6]:
heatmap_hbar(inp_currency="GBP", pairs=PAIRS)